# [Python DB-API module for SQLite 3.](https://docs.python.org/2/library/sqlite3.html)

## [pysqlite Github repository](https://github.com/ghaering/pysqlite)

SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language.  
Some applications can use SQLite for internal data storage.  
It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle.  
The sqlite3 module was written by Gerhard Häring.  
It provides a SQL interface compliant with the DB-API 2.0 specification described by PEP 249.  
To use the module, you must first create a Connection object that represents the database.  
Here the data will be stored in the example.db file:

In [1]:
import sqlite3

conn = sqlite3.connect('example.db'); conn

In [2]:
conn.__doc__

'SQLite database connection object.'

In [3]:
dir(conn)

['DataError',
 'DatabaseError',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'Warning',
 '__call__',
 '__class__',
 '__delattr__',
 '__doc__',
 '__enter__',
 '__exit__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'commit',
 'create_aggregate',
 'create_collation',
 'create_function',
 'cursor',
 'execute',
 'executemany',
 'executescript',
 'interrupt',
 'isolation_level',
 'iterdump',
 'rollback',
 'row_factory',
 'set_authorizer',
 'set_progress_handler',
 'text_factory',
 'total_changes']

You can also supply the special name `:memory:` to create a database in RAM.  
Once you have a Connection, you can create a Cursor object and call its `execute()` method to perform SQL commands:

In [4]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE stocks(date text, trans text, symbol text, qty real, price real)'''); c

In [5]:
# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05', 'BUY', 'RHAT', 100, 35.14)")

# Save (commit) the changes
conn.commit()

# Close the connection when you are done
# Uncommitted changes will be discarded
conn.close()

Remember, if you just close your database connection without calling `commit()` first, your changes will be lost.  
The saved data is persistent and is available in subsequent sessions:

In [6]:
import sqlite3

conn = sqlite3.connect('example.db')
c = conn.cursor(); c

Usually your SQL operations will need to use values from Python variables.  
You shouldn’t assemble your query using Python’s string operations because doing so is insecure; it makes your program vulnerable to an SQL injection attack.  
Instead, use the DB-API’s parameter substitution.  
Put `?` as a placeholder wherever you want to use a value, and then provide a tuple of values as the second argument to the cursor’s `execute()` method.  
Other database modules may use a different placeholder, such as `%s` or `:1`.

In [7]:
# This is insecure and bad
symbol = 'RHAT'
c.execute("SELECT * FROM stocks WHERE symbol = '%s'" % symbol)
print c.fetchone()

(u'2006-01-05', u'BUY', u'RHAT', 100.0, 35.14)


In [8]:
# Do this instead
t = ('RHAT',)
c.execute('SELECT * FROM stocks WHERE symbol = ?', t)
print c.fetchone()

(u'2006-01-05', u'BUY', u'RHAT', 100.0, 35.14)


In [9]:
# Insert multiple records
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

To retrieve data after executing a SELECT statement, you can either treat the cursor as an iterator, call the cursor’s `fetchone()` method to retrieve a single matching row, or call `fetchall()` to get a list of the matching rows.  
This example uses the iterator form:

In [10]:
for row in c.execute('SELECT * FROM stocks ORDER BY price'):
    print row

(u'2006-01-05', u'BUY', u'RHAT', 100.0, 35.14)
(u'2006-03-28', u'BUY', u'IBM', 1000.0, 45.0)
(u'2006-04-06', u'SELL', u'IBM', 500.0, 53.0)
(u'2006-04-05', u'BUY', u'MSFT', 1000.0, 72.0)


You can build a basic shell for SQLite, for experiments and such:

In [11]:
import sqlite3

# Create a database in RAM
# Using ":memory:" will open a database connection to a database that resides in RAM instead of on disk
con = sqlite3.connect(":memory:")
con.isolation_level = None
# Create a Cursor object
cur = con.cursor()

buffer = ""

print "Enter your SQL commands for this sqlite3 database."
print "To exit, enter a blank line."

while True:
    line = raw_input()
    if line == "":
        break
    buffer += line
    if sqlite3.complete_statement(buffer):
        try:
            buffer = buffer.strip()
            cur.execute(buffer)
            if buffer.lstrip().upper().startswith("SELECT"):
                print cur.fetchall()
        except sqlite3.Error as e:
            print "Error: ", e.args[0]
        buffer = ""
        
con.close()

Enter your SQL commands for this sqlite3 database.
To exit, enter a blank line.



## [Connection Objects](https://docs.python.org/2/library/sqlite3.html#connection-objects)

[**create_function** *(name, num_params, func)*](https://docs.python.org/2/library/sqlite3.html#sqlite3.Connection.create_function)  
Creates a user-defined function that you can later use from within SQL statements under the function name *name*.  
*num_params* is the number of parameters the function accepts, and *func* is a Python callable that is called as the SQL function.  
The function can return any of the types supported by SQLite: unicode, str, int, long, float, buffer and None.  

[**create_aggregate** *(name, num_params, aggregate_class)*](https://docs.python.org/2/library/sqlite3.html#sqlite3.Connection.create_aggregate)  
Creates a user-defined aggregate function.  
The aggregate class must implement a step method, which accepts the number of parameters *num_params*, and a finalize method which will return the final result of the aggregate.  
The finalize method can return any of the types supported by SQLite: unicode, str, int, long, float, buffer and None.

[**enable_load_extension** *(enabled)*](https://docs.python.org/2/library/sqlite3.html#sqlite3.Connection.enable_load_extension)  
This routine allows/disallows the SQLite engine to load SQLite extensions from shared libraries.  
SQLite extensions can define new functions, aggregates or whole new virtual table implementations.  
One well-known extension is the fulltext-search extension distributed with SQLite.  
Loadable extensions are disabled by default because some platforms (notably Mac OS X) have SQLite libraries which are compiled without this feature.  
To get loadable extension support, you must modify `setup.py` and remove the line that sets SQLITE_OMIT_LOAD_EXTENSION.

In [12]:
import os

# Destroy example file
os.remove('example.db')